first the file is a pickle type which means that it can be unpacked
<br>
The main goal i see is to find a representation that has convex clusters as that would be the limitation of attempting a clustering.

<br>

wafer defects Center, Donut, Edge-Loc, Edge-Ring, Loc, Random, Scratch, Near-full, none.

In [ ]:
# import numpy, pandas, sklearn for k-means and pca


import numpy as np
import pandas as pd
import sklearn
import pickle
import matplotlib.pyplot as plt
import umap
import cv2

In [ ]:
# import the data from pickle
src = "./data/LSWMD.pkl"
data=pd.read_pickle(src)

print(data.columns)

def clean(x):
    if(len(x) == 0):
        return ""
    else:
        return x[0][0]

data["trianTestLabel"] = data["trianTestLabel"].apply(clean) # caused slowdown

In [ ]:
# #clean the data
# for index,row in data.iterrows():
#     entry = row["trianTestLabel"]
#     #print(entry)
#     if(len(entry) == 0):
#         entry = ""
#     else:
#         entry = entry[0][0]
    
#     data["trianTestLabel"][index] = entry

first the simplest model would be to try to see if the data is linearly seperable. 
So the first thing to try is to see 2d projections of the data to see if they is some inherent clusters.
<br>

since the task is to find an unsupervised method to cluster even though the data seems to be supervised.
The most naive way would be to try K-means or EM in order to cluster. Since the clustering requires a known number of clusters, the naive way was to use the number of clusters as failure types. 
<br>

seems like the wafer map is an image so a CNN might be needed. that doesn't make sense as that is going the supervised learning route.

what about segmentation of the wafer map?

In [ ]:
print(type(data))
print(data.columns)

In [ ]:
# debugging dataframe
#for index, row in data.iterrows():
   #print(index)
print(data["trianTestLabel"][800000]) #[0],entry[2].shape,entry[5],entry[7],entry[9],entry[13])
#    print("\n") 

the columns of the data are 'waferMap', 'dieSize', 'lotName', 'waferIndex', 'trianTestLabel',
       'failureType'
       
size of the wafer is a consistant 2160 size and 45 by 48 shape. (wrong assumption!!!)

need to fetch an example of each type of defect 


In [ ]:
num = 38
# 38(edge-loc),40(loc),44(center),46(none), 161(), 186(edge-ring)
a = data["waferMap"][num]
b = data["failureType"][num]
plt.imshow(a, cmap='hot', interpolation='nearest')
plt.show()
print(a.shape,b)

based on the cell it seems like a good wafer is all zeros

edge-loc seems to be both 


it seems like the cluster space for some cases is not linear. Thus PCA wouldn't be a good represenation either as it depends on linear eigenvalues.
<br>
maybe they could be a meaningful cluster representation of using U-Map or tsne which are non-linear embeddings

The noise might need to have a salt and pepper filter.
<br>

the wafers are of different sizes

In [ ]:
# resize wafermaps for uniformity (lets try 32 by 32)
subdata = data[data['trianTestLabel'] == "Test"] #data cleaning of [["Training"]]-> "Training" needed

waferdata = subdata["waferMap"]

size = (32,32)
newwaferdata = []
for idx,wafer in enumerate(waferdata):
    wafer = cv2.medianBlur(wafer, 3)
    wafer = cv2.resize(wafer, dsize=size, interpolation=cv2.INTER_CUBIC)
    #print(wafer.shape)
    #wafer = np.array(wafer).reshape(size[0]*size[1])
    newwaferdata.append(wafer)

newwaferdata = np.array(newwaferdata)
print(newwaferdata.shape,newwaferdata[0].shape)

#waferarr = np.stack(s.values)

#print(waferarr.shape)

# print(type(waferarr),waferdata.shape, waferarr.shape)
# size = (32, 32)
# length = data.shape[0]
# for idx,wafer in enumerate(waferdata):
#     wafer = cv2.resize(wafer, dsize=size, interpolation=cv2.INTER_CUBIC)
#     waferarr[idx] = np.array(wafer).reshape(size[0]*size[1])
    
# waferarr.reshape(length,size[0]*size[1])
    
# print(waferdata.shape, waferdata[0].shape)    

In [ ]:
num = 186
# 38(edge-loc),40(loc),44(center),46(none), 161(), 186(edge-ring), 811427(scratch), 811448(Edge-Ring)
a = newwaferdata[num]
b = data["failureType"]
plt.imshow(a, cmap='hot', interpolation='nearest')
plt.show()
print(a.shape)

#the median filter removed the salt and pepper noise
# edge ring seems to lose form under the 5 median filter

In [ ]:
# salt and pepper filter

In [ ]:
# umap
length = data.shape[0]
#print(type(waferarr), waferarr[200].shape)
#waferdata.shape = (length, -1) #np.array(waferdata).reshape((length,size[0]*size[1]))

print(length,newwaferdata.shape)
fit = umap.UMAP()
umapdata = fit.fit_transform(newwaferdata)
#811456
# try a subset, which means the test set would be useful to fetch the subset while having each type of wafer

In [ ]:
plt.scatter(umapdata[:,0], umapdata[:,1], c=newwaferdata)
plt.title('UMAP embedding of random colours');

In [ ]:
# k-means
# because of the catagories, assume 9 clusters
kmeans = KMeans(n_clusters=9, random_state=0)
kmeans.fit(umapdata)

sources used
https://docs.python.org/3/library/pickle.html
https://stackoverflow.com/questions/33127866/how-to-pickle-and-unpickle


error with pickle
https://stackoverflow.com/questions/37371451/importerror-no-module-named-pandas-indexes

dacs for dataframe
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html


https://stackoverflow.com/questions/16476924/how-to-iterate-over-rows-in-a-dataframe-in-pandas


the wafermap needs visualization of a heat map
https://stackoverflow.com/questions/33282368/plotting-a-2d-heatmap-with-matplotlib

check umap representation: https://umap-learn.readthedocs.io/en/latest/

get the pandas dataframe row to numpy for umap to work https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_numpy.html

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.to_numpy.html
https://numpy.org/doc/stable/reference/generated/numpy.reshape.html
https://stackoverflow.com/questions/15943769/how-do-i-get-the-row-count-of-a-pandas-dataframe

rescale wafermap
https://stackoverflow.com/questions/48121916/numpy-resize-rescale-image

https://stackoverflow.com/questions/9057379/correct-and-efficient-way-to-flatten-array-in-numpy-in-python


fix shape issue
https://stackoverflow.com/questions/48823400/pandas-series-to-2d-array
https://towardsdatascience.com/get-into-shape-14637fe1cd32
https://stackoverflow.com/questions/40824601/how-to-convert-a-series-of-arrays-into-a-single-matrix-in-pandas-numpy


median filter
https://medium.com/@florestony5454/median-filtering-with-python-and-opencv-2bce390be0d1 
https://docs.opencv.org/master/d4/d13/tutorial_py_filtering.html


sklearn k-means

https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html

dataframe fetch test subset
https://stackoverflow.com/questions/36307609/how-can-i-select-data-from-a-table-with-specific-attribute-with-pandas 

dataframe reading bug (might need to use apply)
https://www.kite.com/python/answers/how-to-modify-all-the-values-in-a-pandas-dataframe-column-in-python